In [29]:
using SparseArrays
using LinearAlgebra
using Statistics
using SuiteSparse
using Plots
E0 = 1;
Emin = 1e-9;
nu = 0.3;
using JuMP
using NLopt
using GLPK
using Optim
function top88mma(nelx,nely,volfrac=0.5,penal=3,rmin=2.5,ft=1)
        
    A11 = [12  3 -6 -3;  3 12  3  0; -6  3 12 -3; -3  0 -3 12]
    A12 = [-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6]
    B11 = [-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4]
    B12 = [ 2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2]
    KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11])

    nodenrs = reshape(1:(1+nelx)*(1+nely),1+nely,1+nelx); #number of the nodes in columns
    edofVec = reshape(2*nodenrs[1:end-1,1:end-1].+1,nelx*nely,1) ;#1st dof of each element (x top left)
    edofMat = zeros(nelx*nely, 8); #every line i contains the 8 dof of the ith element
    noeudsvoisins = [0 1 2*nely.+[2 3 0 1] -2 -1];
    for i = 1:8
        for j = 1:nelx*nely
            edofMat[j,i]= edofVec[j]+ noeudsvoisins[i] ;
        end
    end
    iK = reshape(kron(edofMat,ones(8,1))',64*nelx*nely,1);# line to build  K
    jK = reshape(kron(edofMat,ones(1,8))',64*nelx*nely,1);# columns
    F = spzeros(2*(nely+1)*(nelx+1),1); F[2,1] = -1;
    U = zeros(2*(nely+1)*(nelx+1),1);
    fixeddofs = union(1:2:2*(nely+1),[2*(nelx+1)*(nely+1)]);
    alldofs = 1:2*(nely+1)*(nelx+1);
    freedofs = setdiff(alldofs,fixeddofs);
    iH = ones(nelx*nely*(2*(convert(Int64,ceil(rmin))-1)+1)^2);#lines H
    jH = ones(size(iH));# columns
    sH = zeros(size(iH));# values
    k = 0;
    xnew = zeros(nely,nelx)
    for i1 = 1:nelx
        for j1 = 1:nely
            e1 = (i1-1)*nely+j1;
            for i2 = max(i1-(ceil(rmin)-1),1):min(i1+(ceil(rmin)-1),nelx)
                for j2 = max(j1-(ceil(rmin)-1),1):min(j1+(ceil(rmin)-1),nely)
                    e2 = (i2-1)*nely+j2;
                    k = k+1;
                    iH[k] = e1;
                    jH[k] = e2;
                    sH[k] = max(0,rmin-sqrt((i1-i2)^2+(j1-j2)^2));
                end
            end
        end
    end
    H = sparse(iH,jH,sH);# matrix of the Hei
    Hs = [sum(H[i,:]) for i = 1:(size(H)[1])];# sum of the Hei for e fixed -> coeffs used to divide in the filter
    x = volfrac*ones(nely,nelx);
    xPhys = x;#directly modified in args of mma ? semble inutile avec JuMP
    loop = 0;
    change = 1;
    while change > 0.01
        loop = loop + 1;
        sK = [j*((i+Emin)^penal) for i in ((E0-Emin)*xPhys[:]') for j in KE[:] ];#new values EF
        K = sparse(iK[:],jK[:],sK); K = (K+K')/2;# force to be symmetric
        KK = cholesky(K[freedofs,freedofs]);
        U[freedofs] = KK\(KK'\F[freedofs]);
        
        edM = [convert(Int64,i) for i in edofMat] ;#to index by int
        mat = (U[edM]*KE).*U[edM];
        ce = reshape([sum(mat[i,:]) for i = 1:(size(mat)[1])],nely,nelx);
        c = sum(sum((Emin*ones(size(xPhys)).+(xPhys.^penal)*(E0-Emin)).*ce));
        dc = -penal*(E0-Emin)*xPhys.^(penal-1).*ce;
        dv = ones(nely,nelx);
        
        #m = Model(solver=NLoptSolver(algorithm=:LD_MMA))
        m = Model(with_optimizer(GLPK.Optimizer))
        @variable(m, 0.001<=x1[1:nelx*nely]<=1)
        """
        for i = 1:nelx*nely
            setvalue(x1[i],x[i])
        end
        """
        @NLconstraint(m, sum(x1[i] for i in 1:nelx*nely) <= volfrac*nelx*nely)
        """
        if ft == 1
            dc[:] = H*(x[:].*dc[:])./Hs./max(1e-3,maximum(x[:]));
            function f1(x)
                sum(sum((Emin*ones(size(x)).+(x.^penal)*(E0-Emin)).*ce))
            end

            function f(x, grad)
                if length(grad) > 0
                    grad[:]=H*(x[:].*dc[:])./Hs./max(1e-3,maximum(x[:])); #utiliser automatic differentaion après
                end
                return sum(sum((Emin*ones(size(x)).+(x.^penal)*(E0-Emin)).*ce));
            end

            @objective(m, Max, f1)
        elseif ft == 2

            function f(x, grad)
                if length(grad) > 0
                    grad[:]=H*(x[:].*dc[:])./Hs
                end
                return sum(sum((Emin*ones(size(x)).+(x.^penal)*(E0-Emin)).*ce));
            end


            #dv[:] = H*(dv[:]./Hs);# useless here ?
        end
        """

        @objective(m, Max, sum(sum((Emin*ones(size(x)).+(x.^penal)*(E0-Emin)).*ce)))
        optimize!(m)
        xnew = value.(x1)
        change = maximum(abs.(x-xnew));
        x = xnew ;
        println("loop = ", loop, "Change = ", change ," c = ", c , " densité struct = ", mean(x) )
        heatmap(x)
        if loop == 30
            return heatmap(x)
        end
    end
    return heatmap(x)
end

top88mma (generic function with 5 methods)

In [30]:
top88mma(20,10,0.5,3,2.5,1)

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top88mma(::Int64, ::Int64, ::Float64, ::Int64, ::Float64, ::Int64) at In[29]:77
└ @ Main .\In[29]:77


UndefVarError: UndefVarError: optimize! not defined

In [20]:
import Pkg
Pkg.add("Optim")

   Updating registry at `C:\Users\robin\.julia\registries\General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %.0 %                                  ]  12.9 %.3 %                            ]  27.7 %>                        ]  39.1 %                    ]  48.0 %Fetching: [=======================>                 ]  56.0 %===============================>         ]  76.0 %Fetching: [===================================>     ]  87.4 %> ]  97.1 %

  Resolving package versions...
  Installed PositiveFactorizations ─ v0.2.3
  Installed Compat ───────────────── v3.8.0
  Installed FillArrays ───────────── v0.8.6
  Installed NLSolversBase ────────── v7.6.1
  Installed ArrayInterface ───────── v2.6.2
  Installed FiniteDiff ───────────── v2.3.0
  Installed LineSearches ─────────── v7.0.1
  Installed Optim ────────────────── v0.20.5
   Updating `C:\Users\robin\.julia\environments\v1.4\Project.toml`
  [429524aa] + Optim v0.20.5
   Updating `C:\Users\robin\.julia\environments\v1.4\Manifest.toml`
  [4fba245c] + ArrayInterface v2.6.2
  [34da2185] + Compat v3.8.0
  [1a297f60] + FillArrays v0.8.6
  [6a86dc24] + FiniteDiff v2.3.0
  [d3d80556] + LineSearches v7.0.1
  [d41bc354] + NLSolversBase v7.6.1
  [429524aa] + Optim v0.20.5
  [85a6dd25] + PositiveFactorizations v0.2.3
  [1a1011a3] + SharedArrays 
